In [1]:
%cd ..

/home/crocoder/Desktop/ToxicSpansDetection/toxic-spans-detection


In [2]:
import string
import numpy as np
import unicodedata
from baselines.models import RNNSL
from baselines.spacy_tagging import read_datafile
from evaluation.semeval2021 import f1

In [3]:
train_file = './data/modified_train.csv'
test_file = './data/tsd_trial.csv'

In [4]:
train = read_datafile(train_file)

In [5]:
test = read_datafile(test_file)

In [6]:
spans = train[4937][0]
text = train[4937][1]

In [7]:

def convert_spans_to_token_labels(text,spans):
    token_labels = []
    i=0
    text = ' '.join(text.split())
    while i<len(text):
        if text[i] in string.whitespace:
            i+=1
            continue
        else:
            # print(i,text[i])
            if i in spans:
                token_labels.append(2)
            else:
                token_labels.append(1)
            while(i<len(text) and (text[i] not in string.whitespace)):
                i+=1                
    assert len(text.split())==len(token_labels)
    return token_labels


In [8]:
len(convert_spans_to_token_labels(text,spans))

22

In [9]:
train_token_labels = np.array([convert_spans_to_token_labels(text,spans) for spans,text in train])

In [10]:
test_token_labels = np.array([convert_spans_to_token_labels(text,spans) for spans,text in test])

In [11]:
train_tokens = np.array([text.split() for spans,text in train])

In [12]:
test_tokens = np.array([text.split() for spans,text in test])

In [13]:
rnnsl = RNNSL()

In [14]:
from keras.utils import to_categorical

In [15]:
test_token_labels_oh = [to_categorical(test_token_label, num_classes=3) for test_token_label in test_token_labels]

In [16]:
train_token_labels_oh = [to_categorical(train_token_label,num_classes=3) for train_token_label in train_token_labels]

In [17]:
rnnsl.fit(train_tokens,train_token_labels_oh,validation_data=(test_tokens,test_token_labels_oh))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 200)          7919000   
_________________________________________________________________
dropout (Dropout)            (None, 128, 200)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 400)          641600    
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 3)            1203      
Total params: 8,561,803
Trainable params: 8,561,803
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
 23/248 [=>............................] - ETA: 22:24 - loss: 0.1047 - accuracy: 0.7491 - precision:

KeyboardInterrupt: 

In [21]:
for i in range(len(test_tokens)):
    if len(test_tokens[i])!=len(test_token_labels[i]):
        print("Sorry",i)

In [22]:
from sklearn.metrics import f1_score
def evaluator(p,g):
    # print(len(p),len(g))
    return f1_score(g,p)
val_data = (test_tokens,test_token_labels)
rnnsl.tune_threshold(val_data,evaluator)

 37
23 23
7 7
11 11
14 14
18 18
6 6
12 12
35 35
11 11
33 33
28 28
26 26
110 110
3 3
47 47
10 10
4 4
41 41
76 76
7 7
4 4
37 37
34 34
30 30
21 21
10 10
4 4
67 67
14 14
26 26
29 29
19 19
42 42
15 15
111 111
15 15
10 10
50 50
128 128
13 13
3 3
104 104
8 8
87 87
13 13
29 29
6 6
26 26
16 16
12 12
109 109
10 10
16 16
89 89
35 35
77 77
34 34
26 26
12 12
2 2
26 26
87 87
8 8
9 9
22 22
89 89
14 14
15 15
116 116
80 80
25 25
84 84
33 33
5 5
66 66
7 7
61 61
77 77
39 39
24 24
33 33
16 16
24 24
4 4
16 16
59 59
7 7
62 62
39 39
104 104
26 26
15 15
5 5
90 90
19 19
9 9
4 4
57 57
35 35
11 11
5 5
24 24
95 95
11 11
16 16
17 17
14 14
7 7
23 23
24 24
85 85
5 5
81 81
2 2
31 31
32 32
128 128
26 26
24 24
128 128
19 19
56 56
128 128
21 21
128 128
49 49
6 6
26 26
2 2
9 9
16 16
48 48
13 13
28 28
19 19
47 47
55 55
18 18
46 46
9 9
62 62
1 1
78 78
32 32
77 77
18 18
2 2
24 24
4 4
20 20
50 50
117 117
2 2
21 21
41 41
4 4
33 33
10 10
6 6
28 28
35 35
11 11
60 60
14 14
21 21
35 35
69 69
10 10
70 70
75 75
28 28
45 45
91 91
11

In [26]:
print(rnnsl.threshold)

0.99


In [23]:
predictions = rnnsl.get_toxic_offsets(val_data[0])

In [27]:
print(np.mean([f1(predictions[i],val_data[1][i][:128]) for i in range(len(val_data[1]))]))

0.7009661835748794
